## Wind Site Analysis, Week 2: Optimizing energy extraction with a yaw system

 ### ENRG 421, Spring 2022

I. <font color=green>***Introduction***<font color=black>

In this second part of the wind site analysis lab, we will focus on using the wind speed and directionality data to look at how rotating the turbine to align with the wind impacts the extracted energy. <br><br>

This lab builds on the data imported in the Week 1 lab. You are welcome to work with your data from Week 1. Alternatively, whether you ran into roadblocks getting the data into a usable form or just would prefer to work with a different data set, you may use the data in the [csv file here](https://github.com/timkowalczyk/enrg421/blob/master/keweenawwpa-tdk.csv) which was modified from the Excel sheet from the Keweenaw Bay Indian Community.[[1]](#refs)

Let's kick off with a reminder of the goals for Weeks 1 and 2 on Wind Site Analysis:


II. <font color=green>***Goals***<font color=black>
* To gain experience wrangling raw open-source data for importing into Python
* To produce accurate Weibull-type wind speed distributions from anemometer data
* To estimate the power that could be generated from a turbine sited in the survey area
* To perform a cost/benefit analysis of an active yaw system that turns the turbine into the wind

III. <font color=green>***Importing the wind speed data and Weibull distribution function***<font color=black>

The following code snippet will import the data from the modified Keweenaw Bay Indian Community csv file and associated headers in a form convenient for our use in this lab. It also introduces a suitable Weibull function to get us to where last week's notebook left off.<br>

This is a good opportunity to mention and show a few Python best practices as we merge these code snippets from last week together:
* Where possible, collect all statements **importing** functions from other libraries/modules and place them at the top. This helps other users see what external tools you're using so that they can quickly identify if they are missing any libraries or modules that they need to run the code. (By the way, this is one of the main reasons we're using Anaconda: it has all of the common science-oriented packages installed by default).
* Define **auxiliary functions** (like `F_Weibull()`) before the main code; this practice helps ensure that the function is properly defined before it is called and makes it easier for readers to follow the logical flow.
* The auxiliary function leads off with an extended comment, also called a **docstring** (enclosed in three single-quote marks) documenting what the function returns and what information it needs to do its job.
* Use descriptive variable names so that readers can quickly identify the intended meaning/use of variables in the code. 

In [1]:
from csv import reader
from numpy import array, exp, linspace, power
import matplotlib.pyplot as plt

def F_Weibull(v,k,c):
    ''' Returns the Weibull probability density for wind speed v,
        given the shape factor k and scale factor c'''
    return (k/c)*power(v/c,k-1)*exp(-(power(v/c,k)))

D = [[]] # Empty array to store data
datafilename = 'keweenawwpa-tdk.csv' # This file needs to be downloaded and saved
                                     # in the same directory that you run the notebook from
# Import the data
with open(datafilename) as csv_file:
    csv_reader = reader(csv_file)
    linecounter = 0
    for row in csv_reader:
        data = [float(x) for x in row] # Convert data from strings to numbers
        D.append(data)
csv_file.close()
D = array(D[1:]) # Trims empty first row, converts to numpy array

windspeeds  = D[:,0] # Stores first column of data (without empty first row) as wind speeds
frequencies = D[:,2:]   # Stores remaining columns of data (without first row) as frequencies
directions  = ["Sum","N","NNE","NE","ENE","E","ESE","SE","SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]
norm = sum(frequencies[:,0]) # Total number of wind speed measurements, for normalizing the distribution

# Prepare to plot the "Sum" column of the wind speed data;
# We need to **normalize** this data so that the area under the histogram is 1
# by dividing by total number of measurements in order to compare it to F_Weibull().
x = windspeeds
y = (1/norm)*frequencies[:,0]    # Normalize the "Sum" column of wind speed frequencies

# Prepare to plot the Weibull distribution with best-guess values for the parameters;
# I found k = 1.5, c = 4.0 gives a curve that looks sort of close to the bar chart
k, c = [1.5, 4.0]       # Guess for Weibull distribution parameters
v = linspace(0,25,1000) # Domain for plotting Weibull distribution
w = F_Weibull(v,k,c)    # Produce Weibull distribution

# Show the overlayed plot
fig, axes = plt.subplots(tight_layout=True)
axes.plot(v,w,color='Purple',linewidth=2)
axes.bar(x,y,color='DarkCyan')
plt.xlabel('$v$ (m/s)')
plt.ylabel('$F(v)$')
plt.show()


<Figure size 640x480 with 1 Axes>

IV. <font color=green>***Optimizing the Weibull parameters by nonlinear least-squares fitting***<font color=black>

Here is where the versatility of Python really starts to show an advantage over working with this wind speed data in a spreadsheet application like Excel. We want to find the shape and scale factors $k$ and $c$ that best match the wind speed data in the histogram. Excel can perform linear regression, aka linear least-squares fitting. It can also perform curve-fitting for some simple nonlinear functions like exponentials, logarithms, and power functions. Excel isn't set up to optimize functions more general than these, so at this point we would be stuck unless we can find a plug-in extension that meets our specific goal.<br>

In Python, we can take advantage of numerical optimization tools built into Python's standard scientific library, `scipy` (pronounced "Sigh-pie"). Specifically, scipy contains an `optimize` module with functions that can handle nonlinear least-squares fitting on arbitrary functions in a single line. Here we'll use the `curve_fit()` function from this module. Please check out the documentation for this function [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit) and then run the code snippet below.

In [2]:
from scipy.optimize import curve_fit

init_params = array([1.5,4.0])
opt_params = curve_fit(F_Weibull,x,y,init_params)
print(opt_params)

(array([1.09225034, 3.67560726]), array([[ 0.0013782 , -0.00166283],
       [-0.00166283,  0.01785799]]))


<font color=purple>***Questions:***<font color=black>

1. Compare the number of values returned by `curve_fit()` and stored in the variable `opt_params` with what you would have expected to see. How does the *Returns:* part of the documentation for this function (see link in description above the code snippet) explain why `opt_params` has the number of values that it does?

2. What are the optimal values of $k$ and $c$ for your data set?

3. In the cell below, paste (and modify as needed) the code to re-plot the overlayed bar chart and Weibull distribution, this time with the optimal values for $k$ and $c$.

4. Does the fit look better than the initial guess? Comment briefly on the extent of improvement in the fit compared to the initial guess.

5. Finally, let's use the optimized Weibull distribution you found to estimate the power in the wind over a 10 m<sup>2</sup> swept area at your site.<br><br>
Recall that $P = \frac{1}{2}\rho A \langle v^3 \rangle_F$, where the mean cube wind speed obtained over the Weibull distribution $F$ is given by<br><br>
$\langle v^3 \rangle_F = \int_0^\infty v^3 F(v)\,dv$<br><br>
Use $\rho = 1.2$ kg/m$^3$ and $A = 10$ m$^2$ for this calculation.<br><br>
*Hint:* When you convert this integral into a finite sum so that you can evaluate it numerically, there's no need to go out to inifinity; the domain that you used to plot the Weibull distribution will suffice since $F(v) \rightarrow 0$ at the larger values of $v$.

V. <font color=green>***Yaw systems***<font color=black>

Horizontal axis wind turbines (HAWT) operate most efficiently when they are oriented directly into the wind. Because the wind direction is constantly changing, there is no perfect fixed orientation for a HAWT. Instead, most modern turbines have a **yaw system** that allows the orientation of the wind turbine to change as the directionality of the wind varies. [This article](https://iopscience.iop.org/article/10.1088/1742-6596/524/1/012086/pdf) provides a technical but accessible overview of the physical considerations that go into the design of a yaw system.[[2]](#refs)
    
To determine the impact of yaw on the available wind energy, our goal is to compare the power available to a fixed-yaw turbine versus the power available to a turbine with a passive yaw system that rotates into the wind.

<font color=purple>***Questions:***<font color=black>

1. In the cells below, use the wind speed data to calculate and print:<br>
  a. the mean cubic wind speed $\langle v^3 \rangle$ for each direction. Guidance is provided in the first box to get you started; the only line you need to edit is the one with the `<---` arrow in the comments.<br>
  b. the average power $\langle P \rangle = \frac{1}{2}\rho A \langle v^3 \rangle$ for each direction. (Use $\rho = 1.2$ kg/m$^3$ and $A = 10$ m$^2$).<br>

In [10]:
# 1a
from numpy import zeros 
mean_cube_v = zeros(16) # Initializes a length-16 vector of zeros to store the v^3 values

for i in range(1,16): # loop over all 16 directions (1, ..., 16); skipping column zero (sum over directions)
    this_mean_cube_v = 0. # We'll build up the mean_cube_v for direction i one wind speed at a time
    
    # Take weighted average of v^3 with weights:
    # weight for jth wind speed bin = (frequency of jth wind speed bin)/(sum_of_freqs)
    sum_of_freqs = sum(frequencies[:,i])
    for v in range(len(windspeeds)): # loop over all wind speed bins v
        weight = # <--- Complete the formula here for the correct weight!
        this_mean_cube_v += weight*(windspeeds[v]**3)
        #print(weight)
        
    mean_cube_v[i] = this_mean_cube_v
    
for i in range(1,16):
    print("In the {0:3s} direction, the mean cubic wind speed is {1:10.2f}  m^3/s^3".format(directions[i], mean_cube_v[i]))

In the N   direction, the mean cubic wind speed is       3.71  m^3/s^3
In the NNE direction, the mean cubic wind speed is      10.16  m^3/s^3
In the NE  direction, the mean cubic wind speed is      20.82  m^3/s^3
In the ENE direction, the mean cubic wind speed is      92.64  m^3/s^3
In the E   direction, the mean cubic wind speed is     294.66  m^3/s^3
In the ESE direction, the mean cubic wind speed is     411.25  m^3/s^3
In the SE  direction, the mean cubic wind speed is     448.83  m^3/s^3
In the SSE direction, the mean cubic wind speed is     257.71  m^3/s^3
In the S   direction, the mean cubic wind speed is      90.94  m^3/s^3
In the SSW direction, the mean cubic wind speed is      18.85  m^3/s^3
In the SW  direction, the mean cubic wind speed is      17.88  m^3/s^3
In the WSW direction, the mean cubic wind speed is      13.57  m^3/s^3
In the W   direction, the mean cubic wind speed is      18.52  m^3/s^3
In the WNW direction, the mean cubic wind speed is      10.70  m^3/s^3
In the

In [ ]:
# 1b

2. Which direction would be optimal for siting a *fixed-angle* HAWT at this site? Briefly explain how you know.

3. Open question to explore: How much *more* power is made available for extraction by allowing the turbine to yaw? For this final question, I'm most interested in hearing your approach(es) to the problem, so please provide a written explanation of your reasoning regardless of whether it is accompanied by an explicit calculation. For an A-level grade on this notebook, the explanation should lead to a calculation that you then perform in the notebook to arrive at an estimate for the extra available power.

**Congratulations** on completing the Week 4 notebook! Please remember to save all of your answers and edits before uploading your .ipynb file to the Week 4 assignment on Canvas.

<a id='refs'></a>
### References Cited

1. T. Jimenez. "[The Wind Powering America Anemometer Loan Program: A Retrospective](https://pdfs.semanticscholar.org/6d2c/1a0a0fff48a58ee27fc40d4c9ce5fd0d5db4.pdf)". National Renewable Energy Laboratory, Technical Report NREL/TP-7A30-57351, May 2013.
2. Kim, M.-G. and Dalhoff, P. H. "[Yaw Systems for wind turbines - Overview of concepts, current challenges and design methods
](https://iopscience.iop.org/article/10.1088/1742-6596/524/1/012086/pdf)". J. Phys.: Conf. Series **2014**, *524*, 012086.